## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-15-48-48 +0000,bbc,Starmer suspends Labour MPs for discipline bre...,https://www.bbc.com/news/articles/c5y7zqdwzqyo
1,2025-07-16-15-41-00 +0000,wsj,President Trump indicated to Republican lawmak...,https://www.wsj.com/politics/policy/trump-cons...
2,2025-07-16-15-26-45 +0000,bbc,US senators exempt HIV/Aids funding from plann...,https://www.bbc.com/news/articles/c0q8ypew5l0o
3,2025-07-16-15-18-21 +0000,bbc,US deports five 'barbaric' migrants to Eswatini,https://www.bbc.com/news/articles/clyze8mvzdgo
4,2025-07-16-15-07-54 +0000,nyt,The Air India Plane Crash Took His ‘Everything’,https://www.nytimes.com/2025/07/16/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,12
95,ukraine,7
115,inflation,6
81,canada,5
65,israel,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
1,2025-07-16-15-41-00 +0000,wsj,President Trump indicated to Republican lawmak...,https://www.wsj.com/politics/policy/trump-cons...,42
64,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,35
51,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,31
50,2025-07-16-01-00-00 +0000,wsj,The impact of President Trump’s whirlwind six ...,https://www.wsj.com/economy/trump-effect-start...,28
11,2025-07-16-14-03-03 +0000,nyt,Republicans in Congress Shift to Backing Ukrai...,https://www.nytimes.com/2025/07/15/us/politics...,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
1,42,2025-07-16-15-41-00 +0000,wsj,President Trump indicated to Republican lawmak...,https://www.wsj.com/politics/policy/trump-cons...
40,26,2025-07-16-06-35-00 +0000,wsj,U.K. Inflation Heats Up as Bank of England Sta...,https://www.wsj.com/economy/u-k-inflation-heat...
59,19,2025-07-15-20-10-21 +0000,bbc,Three key questions after Afghan data breach s...,https://www.bbc.com/news/articles/c9w12kdg5zko
68,18,2025-07-15-17-49-34 +0000,bbc,Carney says US-Canada trade deal likely to inc...,https://www.bbc.com/news/articles/c0q82qqxegno
29,16,2025-07-16-11-15-42 +0000,nyt,Israel Attacks Near Syria’s Military Headquart...,https://www.nytimes.com/2025/07/15/world/middl...
11,14,2025-07-16-14-03-03 +0000,nyt,Republicans in Congress Shift to Backing Ukrai...,https://www.nytimes.com/2025/07/15/us/politics...
5,13,2025-07-16-15-05-26 +0000,nyt,"As Iran Deports a Million Afghans, ‘Where Do W...",https://www.nytimes.com/2025/07/16/world/middl...
67,12,2025-07-15-18-27-07 +0000,nyt,Canada Wildfire Smoke Triggers Air Quality Ale...,https://www.nytimes.com/2025/07/14/world/canad...
71,12,2025-07-15-16-57-50 +0000,nyt,Netanyahu’s Ultra-Orthodox Allies Threaten His...,https://www.nytimes.com/2025/07/15/world/middl...
64,12,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
